In [1]:
import pandas, numpy

In [2]:
import matplotlib, matplotlib.pyplot
matplotlib.rcParams.update({'font.size':20, 
                            'font.family':'sans-serif', 
                            'xtick.labelsize':16, 
                            'ytick.labelsize':16, 
                            'figure.figsize':(16*(2/3), 9*(2/3)), 
                            'axes.labelsize':20
                           })

In [3]:
results_dir = '/Users/adrian/research/akureyri/results/sleuth_pipeline/'
tpm_file = '/Users/adrian/research/akureyri/results/deseq2_pipeline/DESeq2_TPM_values.tsv'
counts_file = '/Users/adrian/research/akureyri/results/sleuth_pipeline/sleuth_scaled_reads_bygene.csv'
metadata_file = '/Users/adrian/research/akureyri/results/deseq2_pipeline/metadata.tsv'
annotation_file = '/Users/adrian/research/akureyri/results/deseq2_pipeline/annotation.tsv'

tpm_threshold = 2

# read info

In [4]:
metadata = pandas.read_csv(metadata_file, sep='\t')
metadata.head()

,sample,label,time,culture,path
1,test01,RNA_test_1_EKRN230071387-1A,two,2D,/Users/adrian/research/akureyri/results/kallis...
3,test02,RNA_test_2_EKRN230071388-1A,two,3D,/Users/adrian/research/akureyri/results/kallis...
5,test03,RNA_test_3_EKRN230071389-1A,fourteen,2D,/Users/adrian/research/akureyri/results/kallis...
7,test04,RNA_test_4_EKRN230071390-1A,fourteen,3D,/Users/adrian/research/akureyri/results/kallis...
9,test05,RNA_test_5_EKRN230071391-1A,two,2D,/Users/adrian/research/akureyri/results/kallis...


In [5]:
tpm = pandas.read_csv(tpm_file, sep='\t', index_col=0)
tpm.head()

,test01,test02,test03,test04,test05,test06,test07,test08,test09,test10,test11,test12
ENSG00000000003,12.480951,14.500356,12.973101,13.639502,13.056435,15.536032,13.025809,11.380208,14.604471,16.337347,14.475374,13.477217
ENSG00000000005,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
ENSG00000000419,120.981242,107.924893,120.233355,105.070478,117.550878,101.073762,123.991231,93.626046,123.207637,97.723357,93.615215,112.974659
ENSG00000000457,3.624791,5.523426,3.872989,3.323551,2.324639,5.041956,4.175279,4.372463,4.307249,5.125585,4.414247,4.047019
ENSG00000000460,15.407117,10.366913,12.076084,17.697199,11.087450,8.450714,11.056973,21.588369,17.890017,9.149726,18.332927,18.389641


In [6]:
counts = pandas.read_csv(counts_file, sep='\t', index_col=0)
counts.head()

,test01,test02,test03,test04,test05,test06,test07,test08,test09,test10,test11,test12
ENSG00000000003,678.791432,681.307742,803.568534,685.429129,816.856258,710.076498,702.851933,548.873872,742.745362,764.101705,748.354304,656.643982
ENSG00000000005,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
ENSG00000000419,3044.550721,2347.637018,3428.945961,2442.411910,3417.377571,2141.176937,3098.125112,2158.319618,2932.901829,2141.571141,2282.428412,2594.505554
ENSG00000000457,245.640061,323.341843,299.170783,208.122067,181.089037,287.056176,280.682694,261.297709,272.412792,298.062511,283.427388,244.913462
ENSG00000000460,942.896829,548.083002,842.104219,1000.774721,780.245054,434.552463,671.301519,1167.944258,1022.674986,480.956658,1064.543719,1006.420956


In [7]:
annotation = pandas.read_csv(annotation_file, sep='\t')
annotation.head()

,Unnamed: 0,ensembl_transcript_id,ensembl_gene_id,external_gene_name,gene_biotype,description
0,1,ENST00000387314,ENSG00000210049,MT-TF,Mt_tRNA,mitochondrially encoded tRNA-Phe (UUU/C) [Sour...
1,2,ENST00000389680,ENSG00000211459,MT-RNR1,Mt_rRNA,mitochondrially encoded 12S rRNA [Source:HGNC ...
2,3,ENST00000387342,ENSG00000210077,MT-TV,Mt_tRNA,mitochondrially encoded tRNA-Val (GUN) [Source...
3,4,ENST00000387347,ENSG00000210082,MT-RNR2,Mt_rRNA,mitochondrially encoded 16S rRNA [Source:HGNC ...
4,5,ENST00000386347,ENSG00000209082,MT-TL1,Mt_tRNA,mitochondrially encoded tRNA-Leu (UUA/G) 1 [So...


# contrasts

In [12]:
%%time

def volcano_plotter(contrast):

    input_file_name = contrast[0]
    sorting_label_1 = contrast[1]
    sorting_label_2 = contrast[2]

    # define contrast data
    input_file = results_dir + input_file_name
    df = pandas.read_csv(input_file, sep='\t')
    print(df.shape)
    print(df.head())

    # define contrast antidata
    input_file_name = input_file_name.replace('.tsv', '.anti.tsv')
    input_file = results_dir + input_file_name
    anti = pandas.read_csv(input_file, sep='\t')
    print(anti.shape)
    print(anti.head())

    # filter on fold change on estimated counts
    sub = metadata[metadata[sorting_label_1] == sorting_label_2]
    print(sub)
    labels = sub['sample']
    subset_counts = counts[labels]

    a = subset_counts.iloc[:, :3].median(axis=1)
    b = subset_counts.iloc[:, 3:].median(axis=1)
    c = numpy.log2(0.1)
    

    print(labels)
    print(sub)
    print(subset_counts.head())
    

    return None

contrasts = [['effect_time_2D.tsv', 'culture', '2D'], 
             #['effect_time_3D.tsv', 'culture', '3D'],
             #['effect_culture_day2.tsv', 'time', 'two'],
             #['effect_culture_day14.tsv', 'time', 'fourteen']
            ]

for contrast in contrasts:
    print("\t".join(contrast))
    volcano_plotter(contrast)

effect_time_2D.tsv	culture	2D
(143, 6)
         target_id  ext_gene  num_aggregated_transcripts  sum_mean_obs_counts  \
1  ENSG00000168906     MAT2A                           6            33.919473   
2  ENSG00000181019      NQO1                           6            35.162316   
3  ENSG00000135919  SERPINE2                           9            49.972471   
4  ENSG00000161513      FDXR                          12            35.250416   
5  ENSG00000245532     NEAT1                           8            47.532924   

           pval          qval  
1  4.827822e-13  8.318337e-09  
2  8.752320e-12  7.540124e-08  
3  4.044257e-11  2.322751e-07  
4  1.113098e-09  4.794671e-06  
5  1.807912e-08  6.230065e-05  
(17087, 6)
         target_id ext_gene  num_aggregated_transcripts  sum_mean_obs_counts  \
1  ENSG00000143867     OSR1                           3            15.016730   
2  ENSG00000166803    PCLAF                           7            34.513615   
3  ENSG00000151914      DST    